In [17]:
import requests
import base64
import csv
import json

In [18]:
##Insert credentials, get them here https://developer.spotify.com/dashboard

client_id = ''
client_secret = ''

In [19]:
def get_spotify_token(client_id, client_secret):
    # Endpoint for requesting token
    token_url = 'https://accounts.spotify.com/api/token'

    # Base64 encode Client ID and Client Secret
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

    # Headers
    headers = {
        "Authorization": f"Basic {client_creds_b64}"
    }

    # Data payload
    data = {
        "grant_type": "client_credentials"
    }

    # POST request to get the token
    r = requests.post(token_url, headers=headers, data=data)

    if r.status_code not in range(200, 299):
        raise Exception("Could not authenticate client")
        return None

    token_response_data = r.json()
    return token_response_data['access_token']


In [26]:
token = get_spotify_token(client_id, client_secret)
##print("Your Spotify OAuth Token is:", token)


In [21]:
def fetch_playlist_info(query, token, type='playlist', limit=50):
    # Spotify API endpoint for searching
    url = 'https://api.spotify.com/v1/search'

    # Parameters for the search
    params = {
        'q': query,        # Query string (e.g., 'remaster track:Doxy artist:Miles Davis')
        'type': type,      # Type of search (e.g., 'playlist')
        'limit': limit     # Number of results to return
    }

    # Headers with the OAuth token
    headers = {
        'Authorization': f'Bearer {token}'
    }

    # Making the GET request
    response = requests.get(url, headers=headers, params=params)
    
    # Check for successful response
    if response.status_code == 200:
        return response.json()
    else:
        print('Failed to fetch data:', response.status_code)
        return None



In [22]:
# Replace with your actual OAuth token and your search query
oauth_token = token

##Filter by keyword in search query
search_query = 'Release'


# Fetching playlist information
#I can pass the limit param here to get a higher number of playlists
playlist_info = fetch_playlist_info(search_query, oauth_token)
if playlist_info:
    print(playlist_info)
else:
    print("Error fetching playlist information")

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=Release&type=playlist&offset=0&limit=50', 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0DV6daicUpjCJRrNedWdMi'}, 'href': 'https://api.spotify.com/v1/playlists/0DV6daicUpjCJRrNedWdMi', 'id': '0DV6daicUpjCJRrNedWdMi', 'images': [{'height': None, 'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000bebb6c38dfa6c091b36311c9e67c', 'width': None}], 'name': 'Release Radar | Nov 25', 'owner': {'display_name': 'Colin Hake', 'external_urls': {'spotify': 'https://open.spotify.com/user/gotothelight'}, 'href': 'https://api.spotify.com/v1/users/gotothelight', 'id': 'gotothelight', 'type': 'user', 'uri': 'spotify:user:gotothelight'}, 'primary_color': None, 'public': None, 'snapshot_id': 'MTQyMCxmZjhhZGExN2ZlMmM0ZDkxZTAzMGU3NjFiMTA3NmQ0MjMyZDM3YmE0', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/0DV6daicUpjCJRrNedWdMi/tracks', 'total': 34}, 'type'

In [23]:
# Sample JSON data
playlist_info

# Convert to formatted string and print
formatted_json = json.dumps(playlist_info, indent=4)
##print(formatted_json)


# Iterate through each playlist in the items list
for playlist in playlist_info['playlists']['items']:
    print("Playlist Name:", playlist['name'])
    print("Collaborative:", playlist['collaborative'])
    print("Description:", playlist['description'])
    print("External URLs:", playlist['external_urls']['spotify'])
    print("Playlist ID:", playlist['id'])
    
    # Owner information
    owner = playlist['owner']
    print("Owner Name:", owner['display_name'])
    print("Owner URL:", owner['external_urls']['spotify'])

    print("Public:", playlist['public'])
    print("Snapshot ID:", playlist['snapshot_id'])

    # Tracks information
    tracks = playlist['tracks']
    print("Tracks URL:", tracks['href'])
    print("Total Tracks:", tracks['total'])

    print("Type:", playlist['type'])
    print("------------")



Playlist Name: Release Radar | Nov 25
Collaborative: False
Description: 
External URLs: https://open.spotify.com/playlist/0DV6daicUpjCJRrNedWdMi
Playlist ID: 0DV6daicUpjCJRrNedWdMi
Owner Name: Colin Hake
Owner URL: https://open.spotify.com/user/gotothelight
Public: None
Snapshot ID: MTQyMCxmZjhhZGExN2ZlMmM0ZDkxZTAzMGU3NjFiMTA3NmQ0MjMyZDM3YmE0
Tracks URL: https://api.spotify.com/v1/playlists/0DV6daicUpjCJRrNedWdMi/tracks
Total Tracks: 34
Type: playlist
------------
Playlist Name: Released Playlist
Collaborative: False
Description: 
External URLs: https://open.spotify.com/playlist/56lpi9gTElirhoQWRNWT7M
Playlist ID: 56lpi9gTElirhoQWRNWT7M
Owner Name: Rosa Lee Author
Owner URL: https://open.spotify.com/user/0acgroisnxc350kygwftxn696
Public: None
Snapshot ID: ODMsNjRlNTA2Y2RhZjNlODE0NzVmNjViZWU3N2ZlZmRhYzQxNmRkODlhYg==
Tracks URL: https://api.spotify.com/v1/playlists/56lpi9gTElirhoQWRNWT7M/tracks
Total Tracks: 39
Type: playlist
------------
Playlist Name: New Music Friday
Collaborative: Fa

In [25]:


# Define the header for your CSV file
headers = ['Playlist Name', 'Collaborative', 'Description', 'External URLs', 'Playlist ID', 'Owner Name', 'Owner URL', 'Public', 'Snapshot ID', 'Tracks URL', 'Total Tracks', 'Type']

# Open a file to write to, 'playlist_info.csv' for example
with open('Release.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header
    writer.writerow(headers)

    # Iterate through each playlist in the response data
    for playlist in playlist_info['playlists']['items']:
        writer.writerow([
            playlist['name'],
            playlist['collaborative'],
            playlist['description'],
            playlist['external_urls']['spotify'],
            playlist['id'],
            playlist['owner']['display_name'],
            playlist['owner']['external_urls']['spotify'],
            playlist['public'],
            playlist['snapshot_id'],
            playlist['tracks']['href'],
            playlist['tracks']['total'],
            playlist['type']
        ])

print("CSV file created successfully.")


CSV file created successfully.
